## Εργασία 3 ##

Καλωσήρθατε στην τρίτη σας εργασία. Η εργασία αυτή έχει σκοπό να σας βοηθήσει να εμπεδώσετε τα σύνολα μοντέλων.

Στην εργασία αυτή θα πρέπει να συμπληρώσετε κώδικα Python 3 στα σημεία που αναφέρουν # YOUR CODE HERE. Μην τροποποιείτε τον κώδικα που βρίσκεται εκτός αυτών των περιοχών.

Πρωτού παραδόσετε την εργασία σας σιγουρευτείτε ότι ο κώδικας σε όλα τα κελιά τρέχει σωστά. Για το σκοπό αυτό επιλέξτε από το μενού **Χρόνος εκτέλεσης (runtime) -> Επανεκίνηση περιόδου λειτουργίας και εκτέλεση όλων**.

Συμπληρώστε το όνομα (**NAME**) και το **AEM** σας παρακάτω **εντός των διπλών εισαγωγικών**:

In [9]:
NAME = "Χρήστος Καλαϊτζίδης"
AEM = "4165"

**1 (2 μονάδες)** Σε αυτήν την εργασία θα δουλέψουμε με ένα σύνολο δεδομένων δυαδικής ταξινόμησης, από το δημόσιο αποθετήριο δεδομένων και πειραμάτων μηχανικής μάθησης [openml.org](https://www.openml.org/). Μπορείτε να δείτε πως φορτώνουμε σύνολα δεδομένων από το openml.org μέσα από την sklearn μελετώντας την αντίστοιχη [τεκμηρίωση](https://scikit-learn.org/stable/datasets/loading_other_datasets.html#downloading-datasets-from-the-openml-org-repository). Συγκεκριμένα θέλουμε να δουλέψουμε με την έκδοση 1 του Pima Indians Diabetes Database, το οποίο αφορά τη διάγνωση διαβήτη. Χωρίστε το σύνολο δεδομένων σε δεδομένα εκπαίδευσης (X_train, y_train) και ελέγχου (X_test, y_test) σε ποσοστό 70% και 30% αντίστοιχα με τη συνάρτηση train_test_split (τιμή για random_state βάλτε 0).

In [10]:
# YOUR CODE HERE
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# Φόρτωση του dataset Pima Indians Diabetes από το OpenML
diabetes = fetch_openml(data_id=37, as_frame=True)

# Διαχωρισμός χαρακτηριστικών και ετικέτας (στόχου)
X = diabetes.data
y = diabetes.target

# Χωρισμός σε σύνολο εκπαίδευσης και ελέγχου (70% - 30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [11]:
"""Τεστ ορθής ανάγνωσης και διαχωρισμού του συνόλου δεδομένων"""
assert X_train['mass'][0] == 33.6
assert X_test['pedi'][34] == 0.512

**2 (4 μονάδες)** Υλοποιήστε μια ντετερμινιστική εκδοχή της μεθόδου των τυχαίων υποχώρων, η οποία χτίζει τόσα μοντέλα όσες και οι μεταβλητές εισόδου, κάθε ένα από τα οποία αγνοεί και μία διαφορετική μεταβλητή εισόδου. Π.χ. το πρώτο μοντέλο αγνοεί την πρώτη, το δεύτερο αγνοεί τη δεύτερη κτλ. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. 

In [12]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class RandomSubspaceDet:
    def __init__(self, estimator=DecisionTreeClassifier()):
         # YOUR CODE HERE
        self.estimator = estimator
        self.models = []
        self.feature_indices = []

    def fit(self, X_train, y_train):
         # YOUR CODE HERE
        self.models = []
        self.feature_indices = []
        n_features = X_train.shape[1]

        for i in range(n_features):
            # Επιλέγουμε όλα τα χαρακτηριστικά εκτός από το i
            features = [j for j in range(n_features) if j != i]
            self.feature_indices.append(features)

            # Δημιουργούμε νέο εκτιμητή και τον εκπαιδεύουμε
            model = clone(self.estimator)
            model.fit(X_train.iloc[:, features], y_train)
            self.models.append(model)

    def predict(self, X):
         # YOUR CODE HERE
        predictions = []

        # Κάθε μοντέλο προβλέπει χρησιμοποιώντας το υποσύνολο των χαρακτηριστικών του
        for model, features in zip(self.models, self.feature_indices):
            preds = model.predict(X.iloc[:, features])
            predictions.append(preds)

        # Πλειοψηφία ψήφων κατά στήλη
        predictions = np.array(predictions)  # shape: (n_models, n_samples)
        majority_votes = []

        for i in range(predictions.shape[1]):
            # Παίρνουμε τη συχνότερη πρόβλεψη για το δείγμα i
            vals, counts = np.unique(predictions[:, i], return_counts=True)
            majority = vals[np.argmax(counts)]
            majority_votes.append(majority)

        return np.array(majority_votes)



In [13]:
"""Τεστ ορθής υλοποίησης RandomSubspaceDet"""
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

rs = RandomSubspaceDet(estimator=DecisionTreeClassifier(random_state=1))
rs.fit(X_train, y_train)
assert round(accuracy_score(rs.predict(X_test), y_test), 4) == 0.7446

**3 (4 μονάδες)** Υλοποιήστε τη μέθοδο AdaBoost όπως παρουσιάστηκε στο μάθημα. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. Χρησιμοποιήστε την παράμετρο sample_weight της fit του βασικού μοντέλου για να ορίσετε τα βάρη των παραδειγμάτων εκπαίδευσης. 

In [14]:
import numpy as np
from sklearn.base import clone

class AdaBoost:
    def __init__(self, n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1)):
         # YOUR CODE HERE
        self.n_estimators = n_estimators
        self.base_estimator = estimator
        self.models = []
        self.alphas = []

    def fit(self, X_train, y_train):
         # YOUR CODE HERE
        # Μετατροπή των ετικετών σε {-1, +1}
        y_train = np.where(y_train == 'tested_positive', 1, -1)
        n_samples = len(y_train)
        w = np.ones(n_samples) / n_samples  # αρχικά βάρη

        self.models = []
        self.alphas = []

        for _ in range(self.n_estimators):
            model = clone(self.base_estimator)
            model.fit(X_train, y_train, sample_weight=w)

            y_pred = model.predict(X_train)
            err = np.sum(w * (y_pred != y_train)) / np.sum(w)

            # Αποφυγή διαίρεσης με το μηδέν
            if err == 0:
                alpha = 1e10
            elif err >= 0.5:
                continue  # αγνοούμε "κακούς" ταξινομητές
            else:
                alpha = 0.5 * np.log((1 - err) / err)

            # Ενημέρωση βαρών
            w *= np.exp(-alpha * y_train * y_pred)
            w /= np.sum(w)

            self.models.append(model)
            self.alphas.append(alpha)

    def predict(self, X):
         # YOUR CODE HERE
        model_preds = np.array([
            alpha * model.predict(X) for model, alpha in zip(self.models, self.alphas)
        ])
        agg_pred = np.sign(np.sum(model_preds, axis=0))
        return np.where(agg_pred == 1, 'tested_positive', 'tested_negative')


In [15]:
"""Τεστ ορθής υλοποίησης AdaBoost"""
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

ab = AdaBoost(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=2, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.7706 


In [16]:
#Ίδιο αποτέλεσμα παίρνουμε και με την κλάση της sklearn. Ωστόσο υπάρχουν γενικότερα διαφορές στην υλοποίηση της σε σ΄χέση με αυτήν που είδαμε στη θεωρία, όπως η υποστήριξη παραπάνω από 2 κλάσεων και ο διαφορετικός χειρισμός της τυχαιότητας στην εκπαίδευση των επί μέρους μοντέλων.
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ab = AdaBoostClassifier(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=2, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.7706